In [1]:
from deep_translator import GoogleTranslator
import nltk
nltk.download('punkt')
import sys
import pandas as pd
import re
from Levenshtein import distance as levenshtein_distance

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
n= 33

In [28]:
df_idioms = pd.read_csv("hindi_meaning_idioms_csv.csv")
df_idioms

idioms = df_idioms['idiom_tokens'].tolist()


with open('hindi_text_list_100.txt', 'r', encoding='utf-8') as file:
    all_lines = file.readlines()
    text = all_lines[n]
print(text)

hindi_sentences = idioms
text_to_match = text

बिना सोचे-समझे उसने इतना बड़ा निर्णय ले लिया, यह तो अपने पाँव में कुल्हाड़ी मारना है।



In [29]:
def manual_sentence_tokenize(text):
    """
    Tokenizes the input text into sentences.
    
    Args:
    text (str): Input text to tokenize.
    
    Returns:
    list: List of tokenized sentences.
    """
    return re.split(r'(?<!\d)\.|\.\.\.|!|\?|\n', text)

def find_closest_match(hindi_sentences, sentence_to_match):
    """
    Finds the closest match for a given sentence from a list of Hindi sentences
    using the Levenshtein distance, while preserving the order of matching words.
    
    Args:
    hindi_sentences (list): List of Hindi sentences.
    sentence_to_match (str): Sentence to find the closest match for.
    
    Returns:
    tuple: Closest matched sentence and its Levenshtein distance.
    """
    closest_sentence = None
    min_distance = float('inf')

    sentence_to_match_words = sentence_to_match.split()

    for sentence in hindi_sentences:
        sentence_words = sentence.split()

        # Check for order-preserved matching words
        match_indices = [sentence_words.index(word) for word in sentence_to_match_words if word in sentence_words]

        if sorted(match_indices) == match_indices and len(match_indices) >= 2:
            distance = levenshtein_distance(sentence, sentence_to_match)
            if distance < min_distance:
                min_distance = distance
                closest_sentence = sentence

    return closest_sentence, min_distance

def find_closest_matched_sentence(hindi_sentences, sentences_to_match):
    """
    Finds the closest match for each sentence in a list of sentences to match
    from a list of Hindi sentences.
    
    Args:
    hindi_sentences (list): List of Hindi sentences.
    sentences_to_match (list): List of sentences to find matches for.
    
    Returns:
    tuple: Closest matched sentence, original sentence, its Levenshtein distance,
           and the previous sentence.
    """
    closest_match = None
    min_distance = float('inf')

    for i, text_sentence in enumerate(sentences_to_match):
        prev_sentence = sentences_to_match[i - 1] if i > 0 else ""
        closest_sentence, distance = find_closest_match(hindi_sentences, text_sentence)
        if distance < min_distance:
            min_distance = distance
            closest_match = (closest_sentence, text_sentence, min_distance, prev_sentence)

    return closest_match

# Example usage
if __name__ == "__main__":


    sentences_to_match = manual_sentence_tokenize(text_to_match)

    closest_match = find_closest_matched_sentence(hindi_sentences, sentences_to_match)

    try:
        hindi_sentence, text_sentence, distance, prev_sentence = closest_match
        print(f"Closest Matched Sentence: {hindi_sentence}")
        print(f"With Line: {text_sentence}")
        print(f"Levenshtein Distance: {distance}")
        print(f"Context: {prev_sentence}")
    except TypeError:
        print("No close match found")
        sys.exit()

Closest Matched Sentence: अपने पाँव कुल्हाड़ी मार
With Line: बिना सोचे-समझे उसने इतना बड़ा निर्णय ले लिया, यह तो अपने पाँव में कुल्हाड़ी मारना है।
Levenshtein Distance: 62
Context: 


In [30]:
# Function to check if at least two words match
def match_sentences(hindi_sentences, sentences_to_match):
    matched_sentences = []
    for text_sentence in sentences_to_match:
        for sentence in hindi_sentences:
            # Tokenize the sentences into words (simple split by whitespace)
            hindi_words = set(sentence.split())
            text_words = set(text_sentence.split())

            # Find the intersection of words
            common_words = hindi_words.intersection(text_words)

            # Check if there are at least 2 common words
            if len(common_words) >= 2:
                matched_sentences.append((sentence, text_sentence))
                break
    return matched_sentences

# Get the matched sentences
matched_sentences = match_sentences(hindi_sentences, sentences_to_match)

# Print the results
for hindi_sentence, text_sentence in matched_sentences:
    print(f"Matched Sentence: {hindi_sentence}")
    print(f"With Line: {text_sentence}")
    print()


Matched Sentence: अपने पाँव कुल्हाड़ी मार
With Line: बिना सोचे-समझे उसने इतना बड़ा निर्णय ले लिया, यह तो अपने पाँव में कुल्हाड़ी मारना है।



In [23]:
words_to_remove = set(hindi_sentence.split())
new_sentence= ' '.join(word for word in text_sentence.split() if word not in words_to_remove)
context_hindi = ""
context_hindi = prev_sentence + " " +new_sentence
print(f" Context To Translate: {context_hindi}")

english_context = GoogleTranslator(source='hi', target='en').translate(context_hindi)
print(english_context)

idiom_index = idioms.index(hindi_sentence)
print(idiom_index)

 Context To Translate:  उसने उसे के से दिया।
He gave it to her from K.
94


In [24]:
keyword_set = []
keywords = df_idioms["idiom_keywords"].tolist()
keywords_set = keywords[idiom_index]
keywords_set = keywords_set.split()
print(keywords_set)


['peg,', 'tie']
